https://gist.github.com/joelouismarino/a2ede9ab3928f999575423b9887abd14  
https://github.com/jhcha08/Implementation_DeepLearningPaper/blob/master/CNN.%20GoogleNet.ipynb  

googlenet paper code
https://arxiv.org/abs/1409.4842v1

blog
https://mole-starseeker.tistory.com/13



In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, Dense, Dropout, AveragePooling2D, concatenate, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import models

In [2]:
inputs = Input(shape=(224,224,3))

x = Conv2D(filters = 64, kernel_size = (7,7), strides = (2,2), padding = 'same', activation = 'relu')(inputs)

2021-09-05 00:02:48.040686: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-05 00:02:48.766677: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2021-09-05 00:02:48.766719: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 19534 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2021-09-05 00:02:48.767516: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment vari

In [4]:
model = Model(inputs, x)

In [5]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 112, 112, 64)      9472      
Total params: 9,472
Trainable params: 9,472
Non-trainable params: 0
_________________________________________________________________


In [6]:

from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, Dense, Dropout, AveragePooling2D, concatenate, Flatten
from keras.models import Model
from keras import models

def inception(x, c_1, c_2a, c_2b, c_3a, c_3b, c_m):
    
    pre_layer = x
    
    conv1 = Conv2D(filters = c_1, kernel_size = (1,1), strides = (1,1), padding = 'same', activation = 'relu')(pre_layer)
    
    conv2 = Conv2D(filters = c_2a, kernel_size = (1,1), strides = (1,1), padding = 'same', activation = 'relu')(pre_layer)
    conv2 = Conv2D(filters = c_2b, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu')(conv2)
    
    conv3 = Conv2D(filters = c_3a, kernel_size = (1,1), strides = (1,1), padding = 'same', activation = 'relu')(pre_layer)
    conv3 = Conv2D(filters = c_3b, kernel_size = (5,5), strides = (1,1), padding = 'same', activation = 'relu')(conv3)
    
    maxpool = MaxPooling2D(pool_size = (3,3), strides = (1,1), padding = 'same')(pre_layer)
    maxconv = Conv2D(filters = c_m, kernel_size = (1,1), strides = (1,1), padding = 'same', activation = 'relu')(maxpool)
    
    concat = concatenate([conv1, conv2, conv3, maxconv], axis = -1) # [B,W,H,C] 형태로 결과가 나오므로 채널인 'C' 기준으로 concat
    
    return concat

inputs = Input(shape=(224,224,3))

x = Conv2D(filters = 64, kernel_size = (7,7), strides = (2,2), padding = 'same', activation = 'relu')(inputs)
x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'same')(x)

x = Conv2D(filters = 64, kernel_size = (1,1), strides = (1,1), padding = 'valid', activation = 'relu')(x)
x = Conv2D(filters = 192, kernel_size = (3,3), strides = (1,1), padding = 'same', activation = 'relu')(x)
x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'same')(x)

x = inception(x, 64, 96, 128, 16, 32, 32) # 3a
x = inception(x, 128, 128, 192, 32, 96, 64) # 3b

x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'same')(x)

x = inception(x, 192, 96, 208, 16, 48, 64) # 4a
x = inception(x, 160, 112, 224, 24, 64, 64) # 4b

aux0 = AveragePooling2D(pool_size = (5,5), strides = (3,3), padding = 'valid')(x)
aux0 = Conv2D(filters = 128, kernel_size = (1,1), strides = (3,3), padding = 'same')(aux0)
aux0 = Dense(1024, activation = 'relu')(aux0)
aux0 = Dense(1024, activation = 'relu')(aux0)
softmax0 = Dense(1000, activation = 'softmax')(aux0)

x = inception(x, 128, 128, 256, 24, 64, 64) # 4c
x = inception(x, 112, 144, 288, 32, 64, 64) # 4d
x = inception(x, 256, 160, 320, 32, 128, 128) # 4e

aux1 = AveragePooling2D(pool_size = (5,5), strides = (3,3), padding = 'valid')(x)
aux1 = Conv2D(filters = 128, kernel_size = (1,1), strides = (3,3), padding = 'same')(aux1)
aux1 = Dense(1024, activation = 'relu')(aux1)
aux1 = Dense(1024, activation = 'relu')(aux1)
softmax1 = Dense(1000, activation = 'softmax')(aux1)

x = MaxPooling2D(pool_size = (3,3), strides = (2,2), padding = 'same')(x)

x = inception(x, 256, 160, 320, 32, 128, 128) # 5a
x = inception(x, 384, 192, 384, 48, 128, 128) # 5b

x = AveragePooling2D(pool_size=(7,7), strides = (1,1), padding = 'valid')(x)
x = Dropout(0.4)(x)
x = Flatten()(x)

outputs = Dense(1000, activation = 'softmax')(x)

model = Model(inputs = inputs, outputs = [aux0, aux1, outputs])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 64)   4160        max_pooling2d[0][0]              
____________________________________________________________________________________________

In [9]:
9472*3

28416

In [10]:
4160+110784

114944